<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Student Name: Thomas GRAVIER</b> </h4>
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [1]:
!nvidia-smi

Tue Oct 29 15:26:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   35C    P8              1W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

In [3]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [4]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:

            trainable_params += param.numel()
        
        
        # fill the gap: get the number of trainable parameters: trainable_params
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [6]:
print(print_trainable_parameters(model))

trainable params: 136178560 || all params: 315119488 || trainable%: 43.21489631260127
None


## <b>Configuring LoRA:<b>

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["query_key_value"],
    bias="none",
    task_type="CAUSAL_LM" # fill the gaph
)

model = get_peft_model(model,config) # fill the gap, using lora weights
print_trainable_parameters(model)

trainable params: 1081344 || all params: 316200832 || trainable%: 0.34198012483408013


## <b>Test the model before finetuning:<b>

In [8]:
prompt = "<human>: What equipment do I need for rock climbing?  \n <assistant>: , with an empty response from the assistant"   # # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

<human>: What equipment do I need for rock climbing?  
 <assistant>: , with an empty response from the assistant


Question: what does the temperature do in the above cell?  La température est un hyperparamètre de configuration qui contrôle le caractère aléatoire de la sortie du modèle de langage. Une température élevée produit des résultats plus imprévisibles et créatifs, tandis qu'une température basse produit une sortie plus commune et conservatrice. Par exemple, si vous réglez la température sur 0,5, le modèle générera généralement un texte plus prévisible et moins créatif que si vous réglez la température sur 1,0.

In [9]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:623: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_

<human>: What equipment do I need for rock climbing?  
 <assistant>: , with an empty response from the assistant: , I need a pair of climbing shoes, a pair of climbing gloves, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of climbing harness, a pair of cl

## <b>Loading the question/answer dataset from HuggingFace:<b>

In [10]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [11]:
df_train = pd.DataFrame(data["train"])

In [12]:
df_train.iloc[0]['instruction']

'Hi, I want to learn to play horseshoes. Can you teach me?'

In [13]:
def generate_prompt(data_point):
    parts = [
        f"<human>: {data_point['instruction']}",
        f"<assistant>: {data_point['demonstration']}"
    ]
    return "\n".join(parts)


In [14]:
def generate_prompt(data_point):
    parts = [
        "<human>: " + data_point['instruction'],
        "<assistant>: " + data_point['demonstration']
    ]
    return "\n".join(parts)

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)


Map:   0%|          | 0/147706 [00:00<?, ? examples/s]

## <b>Finetuning:<b>

In [15]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.096500
2,2.035700
3,2.727200
4,2.438300
5,2.735300
6,2.566600
7,3.027700
8,2.123400
9,2.836500
10,2.534000


TrainOutput(global_step=200, training_loss=2.0281689393520357, metrics={'train_runtime': 957.6114, 'train_samples_per_second': 0.835, 'train_steps_per_second': 0.209, 'total_flos': 154874506702080.0, 'train_loss': 2.0281689393520357, 'epoch': 0.005416164543078819})

In [16]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

Reusing TensorBoard on port 6008 (pid 18668), started 1:01:24 ago. (Use '!kill 18668' to kill it.)

## <b>Test the model after the finetuning:<b>

In [17]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need for rock climbing?  
 <assistant>: , with an empty response from the assistant, the assistant will not be able to provide you with a specific answer.  however, the assistant can provide you with some general information about what equipment you may need to climb a rock.  the assistant will also be able to provide you with some general information about what equipment you may need to climb a rock.  the assistant will also be able to provide you with some general information about what equipment you may need to climb a rock.  the assistant will also be able to provide you with some general information about what equipment you may need to climb a rock.  the assistant will also be able to provide you with some general information about what equipment you may need to climb a rock.  the assistant will also be able to provide you with some general information about what equipment you may need to climb a rock.  the assistant will also be able to provide you wi

In [18]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question}\n<assistant>: "  # Transforme la question en un prompt avec une réponse vide
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()


In [19]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 

1. Open the video file in your video editing software.
2. Select the video clip you want to edit.
3. Use the video editing software’s tools to make the changes you want.
4. Save the edited video file.



- Do you know the reasons as to why people love coffee so much? 

1. It’s a great way to get in shape.
2. It’s a great way to get in shape without spending a lot of money.
3. It’s a great way to get in shape without spending a lot of money and without having to go to the gym.
4. It’s a great way to get in shape without spending a lot of money and without having to go to the gym and without having to spend a lot of money.
5. It’s a great way to get in shape without spending a lot of money and without having to go to the gym and without having to spend a lot of money.
6. It’s a great way to get in shape without spending a lot of money and without having to go to the gym and without having to spend a lot of money.
7. It’

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [20]:
prompt_2 = "<system> You are a helpful assistant\n<human>: How a house is made ?\n<assistant>: "
print(prompt_2)

<system> You are a helpful assistant
<human>: How a house is made ?
<assistant>: 


In [21]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant
<human>: How a house is made ?
<assistant>: 1. Build a foundation
2. Build walls
3. Build roof
4. Build windows
5. Build doors
6. Build windows
7. Build doors
8. Build windows
9. Build doors
10. Build windows
11. Build doors
12. Build windows
13. Build doors
14. Build windows
15. Build doors
16. Build windows
17. Build doors
18. Build windows
19. Build doors
20. Build windows
21. Build doors
22. Build windows
23. Build doors
24. Build windows
25. Build doors
26. Build windows
27. Build doors
28. Build windows
29. Build doors
30. Build windows
31. Build doors
32. Build windows
33. Build doors
34. Build windows
35. Build
CPU times: total: 16.8 s
Wall time: 17.4 s


## Loading the preference data from Huggingface:

In [22]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [23]:
def preprocess_data_dpo(data_point):
    return {"system": data_point["system"],
    "question": data_point["question"],
    "prompt": f"<system> {data_point['system']} <human>: {data_point['question']}  \n <assistant>: ",
    "chosen": data_point["chosen"],
    "rejected": data_point["rejected"],
    } # fill the gap, using dpo format

data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

In [24]:
print(data_dpo)

Dataset({
    features: ['system', 'question', 'chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [25]:
print(data_dpo)

Dataset({
    features: ['system', 'question', 'chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [26]:
data_dpo[0]

{'system': 'You are a helpful assistant',
 'question': "What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints'?",
 'chosen': 'Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining.',
 'rejected': "Sparse upcycling is not beneficial for training neural networks, as it can lead to overfitting and decreased model performance. According to 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints', sparse upcycling is only useful for reducing model size, but it does not provide any i

## Finetuning

Question: what is beta in dpo_args? 
Dans dpo_args, le paramètre beta contrôle la sensibilité de la fonction de préférence entre deux réponses. Une valeur de beta élevée accentue les différences entre réponses, rendant la préférence plus tranchée, tandis qu'une valeur faible rend la préférence plus douce, stabilisant l'apprentissage.

In [1]:
!pip install transformers==4.45.2


  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.0
    Uninstalling transformers-4.46.0:
      Successfully uninstalled transformers-4.46.0


In [54]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
    beta=0.1
)

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    model=model,
    train_dataset=data_dpo,
    args=training_args,
    tokenizer=tokenizer,
    beta=dpo_args["beta"],
) # fill the gap

model.config.use_cache = False
trainer.train()

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\trl\trainer\dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\trl\trainer\dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\trl\trainer\dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/2179 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,1.075200
2,0.728000
3,1.118200
4,0.867600
5,1.111300
6,1.056200
7,1.091000
8,0.902800
9,0.947200
10,0.722500


TrainOutput(global_step=200, training_loss=0.17563682752021123, metrics={'train_runtime': 2120.2619, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.094, 'total_flos': 0.0, 'train_loss': 0.17563682752021123, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [55]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\DAO.EZSPACE\anaconda3\envs\mva2\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<system> You are a helpful assistant
<human>: How a house is made ?
<assistant>:  A house is a structure designed to provide a living, living, or working environment for human habitation. It typically consists of a foundation, walls, roof, and various components such as floors, windows, doors, and insulation. The design and construction of a house can vary depending on the intended use, age, and location, but generally, it includes a structure, load-bearing components, and a system for maintaining and providing warmth, protection, and comfort.
CPU times: total: 7.78 s
Wall time: 8.3 s


In [56]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question}\n<assistant>: "
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [57]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))

- Do people dream in color or black and white? 

Dreams can be described from various perspectives, including color, black and white, or other visual or emotional aspects. Some researchers have suggested that dreams may be associated with color perception, with some individuals reporting seeing or experiencing colors in a way that is perceived as black and white. Other researchers have suggested that dreams may be associated with black and white, with some individuals reporting seeing or experiencing colors in a way that is perceived as black and white. Additionally, some researchers have suggested that dreams may be associated with other visual or emotional aspects, such as emotional expression or emotional processing.



- Explain the concept of economic policies in simple terms 

Economic policies are strategic decisions made by governments, businesses, or other actors to influence the allocation of resources, income, and wealth within an economy. These policies can include initiati

Is the response improved after DPO?

**Commentaire :**

Après l'exécution du test d'inférence, les réponses fournies par le modèle DPO se révèlent beaucoup plus précises. On observe une amélioration notable dans la qualité des prédictions, avec des réponses qui démontrent une compréhension contextuelle plus fine et une pertinence accrue par rapport aux questions posées.


In [58]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

Rock climbing involves the use of specialized equipment and gear to support and guide climbers through unstable rock surfaces. Some common equipment needed for rock climbing includes: 
- Climbing harness or harness system 
- Rock climbing harness 
- Head protection gear (e.g., helmet, chin strap) 
- Body protection gear (e.g., helmet, wrist guards) 
- Rock climbing shoes or climbing boots 
- Rock climbing poles or climbing boards 
- Rock climbing gear (e.g., crampons, belay devices) 
- Rock climbing gear for hands and feet (e.g., gloves, mitts) 
- Safety gear (e.g., helmet, wrist guards) 
- Safety harness or harness system 
- Safety gear (e.g., helmet, wrist guards) 
- Safety gear (e.g., helmet, wrist guards) 
- Safety gear (e.g., helmet, wrist guards) 
- Safety gear (e.g., helmet, wrist guards) 
- Safety gear (e.g.,


In [59]:
!pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
accelerate                1.0.1
aiohappyeyeballs          2.4.3
aiohttp                   3.10.10
aiosignal                 1.3.1
alphacsc                  0.4.1.dev23
anyio                     4.6.2.post1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
astunparse                1.6.3
async-lru                 2.0.4
attrs                     24.2.0
audioread                 3.0.1
babel                     2.16.0
beartype                  0.14.1
beautifulsoup4            4.12.3
better-abc                0.0.3
bitsandbytes              0.44.1
bleach                    6.1.0
certifi                   2024.8.30
cffi                      1.17.1
chardet                   3.0.4
charset-normalizer        3.4.0
circuitsvis               1.43.2
click                     8.1.7
colorama                  0.4.6
co

In [60]:
!conda list

# packages in environment at C:\Users\DAO.EZSPACE\anaconda3\envs\mva2:
#
# Name                    Version                   Build  Channel
absl-py                   2.1.0                    pypi_0    pypi
accelerate                1.0.1                    pypi_0    pypi
aiohappyeyeballs          2.4.3                    pypi_0    pypi
aiohttp                   3.10.10                  pypi_0    pypi
aiosignal                 1.3.1                    pypi_0    pypi
alphacsc                  0.4.1.dev23              pypi_0    pypi
anaconda_prompt           1.1.0                haa95532_0  
anyio                     4.6.2.post1              pypi_0    pypi
argon2-cffi               23.1.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
arrow                     1.3.0                    pypi_0    pypi
asttokens                 2.4.1                    pypi_0    pypi
astunparse                1.6.3                    pypi_0    pypi
async-lr